In [1]:
import warnings

warnings.filterwarnings("error")

In [2]:
import sys

sys.path.append('../..')

In [3]:
import dill

loaded_data = None

with open('../simulations/data.pkl', 'rb') as f:
    loaded_data = dill.load(f)

keys = loaded_data.keys()
keys.sort()
print keys

['N', 'credit', 'randomization', 'underlyings']


In [4]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1/360.
delta = 15*step

maturity = loaded_data["randomization"]["maturity"]

#Discount

In [5]:
from finance.discountfactor import ConstantRateDiscountFactor 

r = 0.02
discount = ConstantRateDiscountFactor(r)

# Underlying

In [6]:
udlyings = loaded_data["underlyings"]

bm = udlyings[0]
time_grid = bm.time

# Derivative

In [7]:
derivatives_nb = 1

In [8]:
from finance.products.european.assetswap import (
    SwapContract,
)

swap_delta = 0.5

swap_dates = SwapContract.generate_payment_dates(0, maturity, swap_delta)
swap = SwapContract(bm, discount, swap_dates)

price_0 = swap.price(0.)

print swap
print "\nPrice swap at t=0 without 1st coupon = ",price_0

Swap contract of maturity T = 5 years, over S^0 with strike K = 104.560, paying at {0.00, 0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00}

Price swap at t=0 without 1st coupon =  2.2571402976


# Exposure

In [9]:
from risk.exposures import EuropeanQuantileGeomBrownianExposure

quantile_im = 0.80

hist_drift = bm.drifts[0][0]
hist_vol = bm.vols[0][0]

exposure = EuropeanQuantileGeomBrownianExposure(swap, discount, hist_drift, hist_vol)

# Indexes stuffs

## Ids of $B$ and $C$

In [10]:
copula = loaded_data["credit"]["copula"]

bc_subsets_indexes = loaded_data["credit"]["bc_subsets_indexes"]

#bc_ids = [26, 29, 17, 50, 13, 4, 5, 9, 64]
#bc_positions = [-0.36, -0.44, 0.69, 0.34, -0.05, 0.23, 0.09, -0.46, -0.04]
#Spreads [1053, 367, 176, 73, 61, 56, 52, 45, 108]

#bc_ids = [17, 9, 29, 26, 50, 4, 5, 13, 64]
#bc_positions = [0.69, -0.46, -0.44, -0.36, 0.34, 0.23, 0.09, -0.05, -0.04]

bc_ids = [13, 17]
bc_positions = [1, -1]

# Bank id
b_id = 13

# Counterparty id
c_ids = list(set(bc_ids) - set([b_id]))

print "Bank id: %s"%b_id
print "Counterparties id: %s (nb = %s)"%(c_ids, len(c_ids))

Bank id: 13
Counterparties id: [17] (nb = 1)


In [11]:
obligors_nb = len(copula.subsets[bc_subsets_indexes[-1]][0])

print "Obligor numbers: %s"%obligors_nb

Obligor numbers: 125


## Subsets indexes of the copula for $B$ and $C$

In [12]:
b_subsets_indexes = copula.get_indexes_including(b_id)
c_subsets_indexes = []
for c_id in c_ids:
    tmp = copula.get_indexes_including(c_id)
    c_subsets_indexes += tmp
    
b_subsets_indexes.sort()
c_subsets_indexes.sort()
    
print "Subsets that generated default times: %s\n"%bc_subsets_indexes
print "Bank subsets indexes: %s\n"%b_subsets_indexes
print "Counterparties subsets indexes: %s"%c_subsets_indexes

Subsets that generated default times: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129]

Bank subsets indexes: [13, 128, 129]

Counterparties subsets indexes: [17, 126, 127, 128, 129]


# Portfolio construction

In [13]:
import numpy as np

p_fixed = 1.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike*delta_beta_sum)

print "Notional = %s" % notional

Notional = 0.00202010033417


In [14]:
pos = np.zeros(obligors_nb)
for idx, ps in zip(bc_ids, bc_positions):
    pos[idx] = ps
    
positions = pos.reshape(pos.size, 1)

print pos

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0. -1.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [15]:
from finance.portfolio import CSAPortfolio

port = CSAPortfolio(positions, [notional], [swap], [exposure], b_id)

# VM and IM accounts

In [16]:
from ccp.accounts import Accounts
from ccp.states import MembersState

states = MembersState(obligors_nb)
vm_accounts = Accounts(states, derivatives_nb)
im_accounts = Accounts(states, derivatives_nb)

In [17]:
from credit.default_models import StepwiseConstantIntensity

default_proba_models = []

all_ids = list(copula.subsets[bc_subsets_indexes[-1]][0])
for ii in all_ids:
    _model = None
    if ii in c_ids:
        _indexes = copula.get_indexes_including(ii)
        mat_hzrd_rate = dict()
        for m in copula.models[_indexes]:
            for pill, hzrd_rate in zip(m.pillars, m.intensities):
                if pill not in mat_hzrd_rate:
                    mat_hzrd_rate[pill] = 0.
                mat_hzrd_rate[pill] += hzrd_rate
                
        _model = StepwiseConstantIntensity(mat_hzrd_rate.keys(), mat_hzrd_rate.values())
    
    default_proba_models.append(_model)
    
print default_proba_models

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, <credit.default_models.StepwiseConstantIntensity object at 0x0000000003CAB5C0>, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


# Regulatory capital

In [18]:
from risk.basel.capital import CSARegulatoryCapital

recoveries = [0.4 for _ in all_ids]
regul_cap = CSARegulatoryCapital(vm_accounts, im_accounts, port, default_proba_models, recoveries, b_id)

# Funding and capital parameters

In [19]:
lambda_ = 0.
recov_funding = 1.
k_ = 0.1
cap_conf_level = 0.85

#MC loop

In [20]:
from utils import time_offseter

In [21]:
N = loaded_data["N"]
N = 10000

In [22]:
results_idio = {"cva": {c: {"sum": 0., "sum2": 0.} for c in c_ids},
                "dva": {c: {"sum": 0., "sum2": 0.} for c in c_ids},
                "fva": {c: {"sum": 0., "sum2": 0.} for c in c_ids},
                "kva_ccr": {c: {"sum": 0., "sum2": 0.} for c in c_ids},
                "kva_cva": {c: {"sum": 0., "sum2": 0.} for c in c_ids},                
                "kva": {c: {"sum": 0., "sum2": 0.} for c in c_ids}}

results_globs = {"cva": {"sum": 0., "sum2": 0.},
                 "dva": {"sum": 0., "sum2": 0.},
                 "fva": {"sum": 0., "sum2": 0.},
                 "kva_ccr": {"sum": 0., "sum2": 0.},
                 "kva_cva": {"sum": 0., "sum2": 0.},
                 "kva": {"sum": 0., "sum2": 0.}}                 

In [23]:
from scipy.stats import expon

exp_distrib_params = loaded_data["randomization"]["distrib"]
exp_distrib = expon(loc=exp_distrib_params["loc"], scale=exp_distrib_params["scale"])

zetas = loaded_data["randomization"]["zetas"]
default_times_mat = loaded_data["credit"]["default_times"]

In [24]:
import time

tic = time.time()

b_df_times_indexes = [ii for ii, ind in enumerate(bc_subsets_indexes) if ind in b_subsets_indexes]

for i in range(N):
    # Resurrecting the states
    # resets to 0. the values
    # of the VMs and IMs
    states.resurrect_all()
    
    zeta = zetas[i]
    zeta_delta = time_offseter(zeta+delta, time_grid, True)
    
    # We set the simulated underlying here
    # that has the all time grid with
    # mpor = 1./360
    swap.underlying = udlyings[i]
    
    default_times = default_times_mat[i]    
    b_min_df_time = default_times[b_df_times_indexes].min()
    
    cva_, dva_, fva_, kccr_, kcva_, ktot_ = [0. for _ in range(6)]
    
    if zeta <= maturity:
        cap_losses = port.compute_exposure(zeta, risk_period=delta, conf_level=cap_conf_level, from_=b_id)
        
    for c_id in c_ids:
        cp_subsets_indexes = copula.get_indexes_including(c_id)
        c_df_times_indexes = [ii for ii, ind in enumerate(bc_subsets_indexes) if ind in cp_subsets_indexes]
        
        bc_df_times_indexes = np.append(b_df_times_indexes, c_df_times_indexes)
        bc_df_times_indexes = np.sort(np.unique(bc_df_times_indexes))
        
        bc_df_times = default_times[bc_df_times_indexes]
        
        tau = bc_df_times.min()
        bar_tau = min(tau, maturity)
        
        cva__, dva__, fva__, kccr__, kcva__, ktot__ = [0. for _ in range(6)]
        
        del bc_df_times
        
        if bar_tau <= zeta:
            continue
        
        inv_pdf = 1./exp_distrib.pdf(zeta)
        discount_zeta = discount(zeta)
        discount_zeta_delta = discount(zeta_delta)
        
        # VM computations
        b_vm = port.compute_value(zeta, from_=b_id, towards_=c_id)
        
        vm_accounts.put_amounts(b_id, b_vm)
        vm_accounts.put_amounts(c_id, -b_vm)

        # IM computations
        b_im = port.compute_exposure(zeta, risk_period=delta, conf_level=quantile_im, from_=b_id, towards_=c_id)
        c_im = port.compute_exposure(zeta, risk_period=delta, conf_level=quantile_im, from_=c_id, towards_=b_id)
        
        im_accounts.put_amounts(b_id, b_im)
        im_accounts.put_amounts(c_id, c_im)
        
        p_zeta = b_vm.sum()
        
        p_zeta_delta = port.compute_value(zeta_delta, from_=b_id, towards_=c_id).sum()
        # We suppose here that \Delta_{\zeta} is null
        q_zeta_delta = p_zeta_delta
        
        c_min_df_time = default_times[c_df_times_indexes].min()
        
        ####################
        ##### FVA part #####
        ####################
        b_minus_funded_collat = p_zeta + im_accounts.get_amount(b_id).sum() - im_accounts.get_amount(c_id).sum()

        b_gamma_zeta = copula.tot_gamma(zeta, subsets_indexes=b_subsets_indexes)
        
        bar_lambda_zeta = b_gamma_zeta * 0.6
        tilde_lambda_zeta = bar_lambda_zeta - (1.-recov_funding)*b_gamma_zeta
        c_zeta = 0.
        lambda_zeta = lambda_
        
        fva__ += (c_zeta*b_minus_funded_collat + \
                  tilde_lambda_zeta*np.maximum(p_zeta-b_minus_funded_collat, 0.) + \
                  lambda_zeta*np.minimum(p_zeta-b_minus_funded_collat, 0.))
    
        ######################
        ##### K_CCR part #####
        ######################
        kccr__ = k_*regul_cap.compute_kccr(c_id, zeta, losses=cap_losses, use_vm=False)
            
        ######################
        ##### K_CVA part #####
        ######################
        kcva__ = k_*regul_cap.compute_kcva(c_id, zeta, losses=cap_losses, use_vm=False)
        
        ########################
        ##### K_synth part #####
        ########################
        ktot__ = kccr__ + kcva__
        
        for df_time_index in bc_df_times_indexes:
            subset_index = bc_subsets_indexes[df_time_index]
            gamma_zeta = copula.gamma(subset_index, zeta)
            
            ####################
            ##### CVA part #####
            ####################
            c_default_indic = df_time_index in c_df_times_indexes
            if not c_default_indic:
                c_default_indic = c_min_df_time <= zeta_delta
                
            if c_default_indic:
                b_collat = p_zeta + im_accounts.get_amount(b_id).sum()
                loss = np.maximum(q_zeta_delta-b_collat, 0.)
                cva__ += gamma_zeta*(1.-recoveries[c_id])*loss                
                
            ####################
            ##### DVA part #####
            ####################
            b_default_indic = df_time_index in b_df_times_indexes
            if not b_default_indic:
                b_default_indic = b_min_df_time <= zeta_delta
                
            if b_default_indic:
                c_collat = p_zeta - im_accounts.get_amount(c_id).sum()
                loss = -np.minimum(q_zeta_delta-c_collat, 0.)
                dva__ += gamma_zeta*(1.-recoveries[b_id])*loss
                        
        results_idio['cva'][c_id]["sum"] += (inv_pdf*discount_zeta_delta*cva__)
        results_idio['cva'][c_id]["sum2"] += (inv_pdf*discount_zeta_delta*cva__)**2
        
        results_idio['dva'][c_id]["sum"] += (inv_pdf*discount_zeta_delta*dva__)
        results_idio['dva'][c_id]["sum2"] += (inv_pdf*discount_zeta_delta*dva__)**2
        
        results_idio['fva'][c_id]["sum"] += (inv_pdf*discount_zeta*fva__)
        results_idio['fva'][c_id]["sum2"] += (inv_pdf*discount_zeta*fva__)**2
        
        results_idio['kva_ccr'][c_id]["sum"] += (inv_pdf*discount_zeta*kccr__)
        results_idio['kva_ccr'][c_id]["sum2"] += (inv_pdf*discount_zeta*kccr__)**2
        
        results_idio['kva_cva'][c_id]["sum"] += (inv_pdf*discount_zeta*kcva__)
        results_idio['kva_cva'][c_id]["sum2"] += (inv_pdf*discount_zeta*kcva__)**2
        
        results_idio['kva'][c_id]["sum"] += (inv_pdf*discount_zeta*ktot__)
        results_idio['kva'][c_id]["sum2"] += (inv_pdf*discount_zeta*ktot__)**2
        
        cva_ += inv_pdf*discount_zeta_delta*cva__
        dva_ += inv_pdf*discount_zeta_delta*dva__
        fva_ += inv_pdf*discount_zeta*fva__
        kccr_ += inv_pdf*discount_zeta*kccr__
        kcva_ += inv_pdf*discount_zeta*kcva__
        ktot_ += inv_pdf*discount_zeta*ktot__
        
    results_globs["cva"]["sum"] += cva_
    results_globs["cva"]["sum2"] += cva_**2
    
    results_globs["dva"]["sum"] += dva_
    results_globs["dva"]["sum2"] += dva_**2
    
    results_globs["fva"]["sum"] += fva_
    results_globs["fva"]["sum2"] += fva_**2
    
    results_globs["kva_ccr"]["sum"] += kccr_
    results_globs["kva_ccr"]["sum2"] += kccr_**2
    
    results_globs["kva_cva"]["sum"] += kcva_
    results_globs["kva_cva"]["sum2"] += kcva_**2
    
    results_globs["kva"]["sum"] += ktot_
    results_globs["kva"]["sum2"] += ktot_**2

toc = time.time()

#Results

In [25]:
print "Results for %d iterations (%s secs.)"%(N, toc-tic)
print

print "Used discount factor: %s"%discount
print

print "Bank index %i that belongs to the following MO copula subsets:"%b_id
for idx in b_subsets_indexes:
    print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[idx][0]], \
                                                     copula.pillars[idx], copula.intensities[idx])
    
print "Counterparties indexes:"
print c_ids

print "\n-----------------------------------------------------\n"

for idx in c_ids:
    print "Counterparty index %s belongs to the following MO copula subses:"%idx
    sub_indexes = copula.get_indexes_including(idx)
    for subset_idx in sub_indexes:
        print "- %s with pillars %s and intensity %s\n" % ([x for x in copula.subsets[subset_idx][0]], \
                                                           copula.pillars[subset_idx], copula.intensities[subset_idx])        
    print
    
print "\n-----------------------------------------------------\n"
    
print "Derivatives:"
for d in [swap]:
    print "- %s"%d
    
print "\nPositions:"
print port.positions

print "Confidence level used for IM: %.2f"%quantile_im

Results for 10000 iterations (47.5639998913 secs.)

Used discount factor: Constant discount factor process with rate r = 0.02

Bank index 13 that belongs to the following MO copula subsets:
- [13] with pillars [3 5] and intensity [ 0.0067907   0.01187502]

- [0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 113, 114, 116, 118, 119, 120, 122, 124] with pillars [3 5] and intensity [  2.01650000e-04   9.33720000e-05]

- [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104,

In [26]:
from scipy.stats import norm

conf_level = 0.95
z_level = norm.ppf(0.5*(1+conf_level))

print "Results of the individual xVA:\n"
keys = results_idio.keys()
keys.sort()

for k in keys:    
    print "%s:"%k.upper()
    print "---------------------------"
    v = results_idio[k]
    for id_, res_ in v.iteritems():        
        mean__ = res_['sum']/N
        mod_var__ = (res_['sum2']/N - mean__**2) / (N-1.)
        half_inter = z_level*np.sqrt(mod_var__)
        print "Id %s: [%f, %f]"%(id_, mean__-half_inter, mean__+half_inter)
    print

Results of the individual xVA:

CVA:
---------------------------
Id 17: [0.000199, 0.000228]

DVA:
---------------------------
Id 17: [0.000062, 0.000071]

FVA:
---------------------------
Id 17: [0.000095, 0.000146]

KVA:
---------------------------
Id 17: [0.000658, 0.000675]

KVA_CCR:
---------------------------
Id 17: [0.000449, 0.000460]

KVA_CVA:
---------------------------
Id 17: [0.000209, 0.000215]



In [27]:
print "Results of the global xVA:\n"
keys = results_globs.keys()
keys.sort()

excel_results = dict()

for k in keys:
    mean_ = results_globs[k]['sum']/N
    mod_var_ = (results_globs[k]['sum2']/N - mean_**2) / (N-1.)
    half_inter = z_level*np.sqrt(mod_var_)
    
    excel_results[k.upper()] = "%s \t %s"%(mean_, half_inter / mean_)
    
    print "The %s for the bank lies in [%f, %f]"%(k.upper(), mean_-half_inter, mean_+half_inter)

Results of the global xVA:

The CVA for the bank lies in [0.000199, 0.000228]
The DVA for the bank lies in [0.000062, 0.000071]
The FVA for the bank lies in [0.000095, 0.000146]
The KVA for the bank lies in [0.000658, 0.000675]
The KVA_CCR for the bank lies in [0.000449, 0.000460]
The KVA_CVA for the bank lies in [0.000209, 0.000215]


In [28]:
file_ = open('../Bilat 1vs1.txt', 'a')

file_.write("%s vs %s\n"%(b_id, c_ids[0]))

keys = excel_results.keys()
keys.sort()

for k in keys:
    file_.write("%s \t %s\n"%(k, excel_results[k]))
    
file_.write("\n")
file_.close()